In [ ]:
import tensorflow as tf
%load_ext tensorboard.notebook
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Softmax, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import datetime
from utils import *

In [ ]:
train_dir = os.path.join('images', 'train')
valid_dir = os.path.join('images', 'validation')
test_dir = os.path.join('images', 'test')

class_names = get_class_labels() # in the prediction output, the order is point_foreward (0), point_up (1), curl (2)

In [ ]:
batch_size = 16
epochs = 100
HEIGHT=75
WIDTH=75

In [ ]:
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True,
    zoom_range=0.5
)
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=128,
    directory=train_dir,
    shuffle=True,
    target_size=(HEIGHT, WIDTH),
    classes=class_names,
    class_mode='sparse'
)
valid_image_generator = ImageDataGenerator(
    rescale=1./255,
)
valid_data_gen = valid_image_generator.flow_from_directory(
    batch_size=128,
    directory=valid_dir,
    shuffle=True,
    target_size=(HEIGHT, WIDTH),
    classes=class_names,
    class_mode='sparse'
)

In [ ]:
# train_dir = os.path.join('..', 'detect_classify_images', 'failed')

total_train = 0
total_val = 0
images = []
labels = []
for i, direc in enumerate(class_names):
    train_class_dir = os.path.join(train_dir, direc)
    files = os.listdir(train_class_dir)
    total_train += len(files)
    for f in files:
        image = cv2.imread(os.path.join(train_class_dir, f))
        image = image / 255.0
        image = cv2.resize(image, (HEIGHT, WIDTH)) 
        images.append(image)
        labels.append(i)
        
    valid_class_dir = os.path.join(valid_dir, direc)
    files = os.listdir(valid_class_dir)
    total_val += len(files)
        
images = np.array(images)
labels = np.array(labels)

In [ ]:
# model = Sequential([
#     Flatten(input_shape=(HEIGHT, WIDTH, 3)),
#     Dense(128, activation='relu'),
#     Dense(3, activation='softmax'),
# ])

# model = Sequential([
#     # Layer 1
#     Conv2D(64, (5, 5), input_shape=(HEIGHT, WIDTH, 3)),
#     Activation('relu'),
#     MaxPooling2D(pool_size=(3, 3), strides=2),
#     Dropout(0.2),
    
#     # Layer 2
#     Conv2D(64, (5, 5), padding='same'),
#     Activation('relu'),
#     MaxPooling2D(pool_size=(3, 3), strides=2),
    
#     # Layer 3
#     Conv2D(128, (3, 3), padding='same'),
#     Activation('relu'),
#     MaxPooling2D(pool_size=(3, 3), strides=2),
#     Dropout(0.2),
    
#     Flatten(),
#     Dense(512),
#     Activation('relu'),
#     Dense(3),
#     Softmax()
# ])

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(HEIGHT, WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3)
])

model = load_classifier('model3')

In [ ]:
opt = tf.keras.optimizers.Adam(lr=1e-7)
model.compile(optimizer=opt,
              loss=sparse_categorical_crossentropy_with_logits,
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=100,
    validation_data=valid_data_gen,
    validation_steps=total_val // batch_size
)
# history = model.fit(images, labels, epochs=10)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
correct = 0
for i, image in enumerate(images):
    batch = np.expand_dims(image, axis=0).astype(np.float32)
    res = model.predict(batch)
    prediction = np.argmax(res[0])
    if prediction == labels[i]:
        correct += 1
#     if labels[i] == 2:
    print('prediction: {} vs actual: {} | res: {} '.format(prediction, labels[i], res))
print('correct: {} | num labels: {} => {}'.format(correct, len(labels), correct / len(labels)))

In [ ]:
model.save('model3')

In [ ]:
# testing only passed this cell

test_images = []
test_labels = []
for i, direc in enumerate(class_names):
    test_class_dir = os.path.join(test_dir, direc)
    files = os.listdir(test_class_dir)
    for f in files:
        image = cv2.imread(os.path.join(test_class_dir, f))
        image = image / 255.0
        image = cv2.resize(image, (HEIGHT, WIDTH)) 
        test_images.append(image)
        test_labels.append(i)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
correct = 0
for i, image in enumerate(test_images):
    batch = np.expand_dims(image, axis=0).astype(np.float32)
    res = model.predict(batch)
    prediction = np.argmax(res[0])
    if prediction == test_labels[i]:
        correct += 1
    print('prediction: {} vs actual: {}'.format(prediction, test_labels[i]))
print('correct: {} | num labels: {} => {}'.format(correct, len(test_labels), correct / len(test_labels)))

In [ ]:
# model.save('model_simple_data_aug2')
class_labels = get_class_labels()

In [ ]:
# live demo
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    
    crop = frame[200:325, 200:325].copy()
    cv2.rectangle(frame, (200, 200), (275, 275), (0, 255, 0), 3, 1)
    img = cv2.resize(crop, (75, 75))
    batch = np.expand_dims(img, axis=0).astype(np.float32)
    res = model.predict(batch)
    name = class_labels[np.argmax(res[0])]
#     print(name)
    
    cv2.putText(frame, "{} | {}".format(name, res), (40, 40), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()